In [53]:
import pandas as pd
import gzip as gz
import matplotlib as plt
import sqlite3
import os
import zipfile
import seaborn as sns
import math

In [54]:
# Opened the relevant datasets to be explored in jupyter notebook
#if not os.path.exists('zippedData/im.db'):
with zipfile.ZipFile('zippedData/im.db.zip') as my_zip:
    zipfile.ZipFile.extractall(my_zip,path='zippedData/')
with gz.open('zippedData/rt.reviews.tsv.gz') as f:
    rt_reviews = pd.read_csv(f,delimiter='\t',encoding='latin1')
with gz.open('zippedData/rt.movie_info.tsv.gz') as f:
    rt_movie_info = pd.read_csv(f,delimiter='\t',encoding='latin1')
with gz.open('zippedData/tmdb.movies.csv.gz') as f:
    tmdb_movies = pd.read_csv(f)
with gz.open('zippedData/tn.movie_budgets.csv.gz') as f:
    tn_movie_budgets = pd.read_csv(f)
with gz.open('zippedData/bom.movie_gross.csv.gz') as f:
    bom_movie_gross = pd.read_csv(f)

In [55]:
# Connected to and opened the imdb database to utilize the data
im_db = sqlite3.connect('zippedData/im.db')
imdb_df = pd.read_sql("""SELECT * FROM movie_basics""",im_db)

In [56]:
# Created a SQL query to pull director name and info from the imdb database
director_info = """
SELECT
    m.movie_id,
    m.primary_title,
    m.genres,
    d.person_id,
    p.primary_name,
    p.death_year
    
    
FROM
    movie_basics AS m
    JOIN 
        directors AS d ON m.movie_id = d.movie_id
    JOIN
        known_for AS kf ON m.movie_id = kf.movie_id
    JOIN
        persons AS p ON p.person_id = d.person_id
GROUP BY
    m.movie_id, kf.person_id, p.primary_name
;
"""
director_info_result = pd.read_sql(director_info, im_db)
director_info_result

,movie_id,primary_title,genres,person_id,primary_name,death_year
0,tt0063540,Sunghursh,"Action,Crime,Drama",nm0712540,Harnam Singh Rawail,2004.0
1,tt0063540,Sunghursh,"Action,Crime,Drama",nm0712540,Harnam Singh Rawail,2004.0
2,tt0069049,The Other Side of the Wind,Drama,nm0000080,Orson Welles,1985.0
3,tt0069049,The Other Side of the Wind,Drama,nm0000080,Orson Welles,1985.0
4,tt0069049,The Other Side of the Wind,Drama,nm0000080,Orson Welles,1985.0
...,...,...,...,...,...,...
991711,tt9916754,Chico Albuquerque - Revelações,Documentary,nm8349149,Vinicius Augusto Bozzo,NaN
991712,tt9916754,Chico Albuquerque - Revelações,Documentary,nm9272490,Angela Gurgel,NaN
991713,tt9916754,Chico Albuquerque - Revelações,Documentary,nm8349149,Vinicius Augusto Bozzo,NaN
991714,tt9916754,Chico Albuquerque - Revelações,Documentary,nm9272490,Angela Gurgel,NaN


In [57]:
director_info_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991716 entries, 0 to 991715
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   movie_id       991716 non-null  object 
 1   primary_title  991716 non-null  object 
 2   genres         973303 non-null  object 
 3   person_id      991716 non-null  object 
 4   primary_name   991716 non-null  object 
 5   death_year     5020 non-null    float64
dtypes: float64(1), object(5)
memory usage: 45.4+ MB


In [81]:
# Created a new variable that filtered the director_info_results to drop all director's who are deceased
# and to drop any duplicate movie_ids 
clean_director_info = director_info_result[director_info_result['death_year'].isnull()].drop_duplicates(['primary_title'])
clean_director_info[clean_director_info['primary_name'] == 'James Cameron']

,movie_id,primary_title,genres,person_id,primary_name,death_year
76842,tt1630029,Avatar 2,"Action,Adventure,Fantasy",nm0000116,James Cameron,NaN
115933,tt1757678,Avatar 3,"Action,Adventure,Drama",nm0000116,James Cameron,NaN
401367,tt3095356,Avatar 4,"Action,Adventure,Fantasy",nm0000116,James Cameron,NaN


In [82]:
clean_director_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123023 entries, 18 to 991700
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   movie_id       123023 non-null  object 
 1   primary_title  123023 non-null  object 
 2   genres         120459 non-null  object 
 3   person_id      123023 non-null  object 
 4   primary_name   123023 non-null  object 
 5   death_year     0 non-null       float64
dtypes: float64(1), object(5)
memory usage: 6.6+ MB


In [83]:
tn_movie_budgets['year'] = tn_movie_budgets['release_date'].str[-4:]
tn_movie_budgets['month'] = tn_movie_budgets['release_date'].str[:3]
tn_movie_budgets['clean_budget'] = tn_movie_budgets['production_budget'].str.replace('$','')
tn_movie_budgets['clean_budget'] = tn_movie_budgets['clean_budget'].str.replace(',','').astype(int)
tn_movie_budgets['clean_domestic'] = tn_movie_budgets['domestic_gross'].str.replace('$','')
tn_movie_budgets['clean_domestic'] = tn_movie_budgets['clean_domestic'].str.replace(',','').astype(int)
tn_movie_budgets['clean_worldwide'] = tn_movie_budgets['worldwide_gross'].str.replace('$','')
tn_movie_budgets['clean_worldwide'] = tn_movie_budgets['clean_worldwide'].str.replace(',','').astype('int64')
tn_movie_budgets['clean_foreign'] = tn_movie_budgets['clean_worldwide'] - tn_movie_budgets['clean_domestic']
tn_movie_budgets.rename({'movie':'title'},axis=1,inplace=True)
#tn_movie_budgets.info()
tn_relevant_cols = ['title','year','month','clean_budget','clean_domestic','clean_foreign','clean_worldwide']
tn_relevant = tn_movie_budgets[tn_relevant_cols]
#tn_relevant.head()

In [84]:
bom_movie_gross['foreign_gross'] = bom_movie_gross['foreign_gross'].fillna('0')
bom_movie_gross.dropna(inplace=True)
bom_movie_gross['clean_domestic'] = bom_movie_gross['domestic_gross'].astype(int)
bom_movie_gross['clean_foreign'] = bom_movie_gross['foreign_gross'].str.replace(',','').astype(float)
bom_movie_gross['clean_foreign'] = bom_movie_gross['clean_foreign'].apply(math.trunc)
bom_movie_gross['clean_worldwide'] = bom_movie_gross['clean_domestic'] + bom_movie_gross['clean_foreign']
#bom_movie_gross.info()
bom_relevant_cols = ['title','year','clean_domestic','clean_foreign','clean_worldwide']
bom_relevant = bom_movie_gross[bom_relevant_cols]
#bom_relevant.head()

In [85]:
combined_financials = pd.concat([tn_relevant,bom_relevant],ignore_index=True)
combined_financials = combined_financials.drop_duplicates(subset='title',keep='first')
#combined_financials.plot('clean_domestic','clean_foreign',kind='scatter')

#Much of the budget column is empty because bom_movie_gross does not include budget information
#Going to fill the NA budgets with the median value
combined_financials['clean_budget'].fillna(combined_financials['clean_budget'].median(),inplace=True)
combined_financials['roi_domestic'] = 100 * (combined_financials['clean_domestic'] - combined_financials['clean_budget']) / combined_financials['clean_budget']
combined_financials['roi_worldwide'] = 100 * (combined_financials['clean_worldwide'] - combined_financials['clean_budget']) / combined_financials['clean_budget']
combined_financials['clean_profit'] = combined_financials['clean_worldwide'] - combined_financials['clean_budget']
#combined_financials.head()

In [86]:
all_movie_data = clean_director_info.merge(combined_financials, left_on = 'primary_title',right_on ='title', how = 'inner')
all_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3667 entries, 0 to 3666
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         3667 non-null   object 
 1   primary_title    3667 non-null   object 
 2   genres           3656 non-null   object 
 3   person_id        3667 non-null   object 
 4   primary_name     3667 non-null   object 
 5   death_year       0 non-null      float64
 6   title            3667 non-null   object 
 7   year             3667 non-null   object 
 8   month            2260 non-null   object 
 9   clean_budget     3667 non-null   float64
 10  clean_domestic   3667 non-null   int32  
 11  clean_foreign    3667 non-null   int64  
 12  clean_worldwide  3667 non-null   int64  
 13  roi_domestic     3667 non-null   float64
 14  roi_worldwide    3667 non-null   float64
 15  clean_profit     3667 non-null   float64
dtypes: float64(5), int32(1), int64(2), object(8)
memory usage: 4

In [87]:
all_movie_data.head()

,movie_id,primary_title,genres,person_id,primary_name,death_year,title,year,month,clean_budget,clean_domestic,clean_foreign,clean_worldwide,roi_domestic,roi_worldwide,clean_profit
0,tt0249516,Foodfight!,"Action,Animation,Comedy",nm0440415,Lawrence Kasanoff,NaN,Foodfight!,2012,Dec,45000000.0,0,73706,73706,-100.000000,-99.836209,-44926294.0
1,tt0293429,Mortal Kombat,"Action,Adventure,Fantasy",nm2585406,Simon McQuoid,NaN,Mortal Kombat,1995,Aug,20000000.0,70433227,51700000,122133227,252.166135,510.666135,102133227.0
2,tt0315642,Wazir,"Action,Crime,Drama",nm2349060,Bejoy Nambiar,NaN,Wazir,2016,NaN,17000000.0,1100000,0,1100000,-93.529412,-93.529412,-15900000.0
3,tt0326592,The Overnight,None,nm1208371,Jed I. Goodman,NaN,The Overnight,2015,Jun,200000.0,1109808,56188,1165996,454.904000,482.998000,965996.0
4,tt0337692,On the Road,"Adventure,Drama,Romance",nm0758574,Walter Salles,NaN,On the Road,2013,Mar,25000000.0,720828,8592474,9313302,-97.116688,-62.746792,-15686698.0


In [89]:
all_movie_data[all_movie_data['primary_title'] == 'Avatar 2']

,movie_id,primary_title,genres,person_id,primary_name,death_year,title,year,month,clean_budget,clean_domestic,clean_foreign,clean_worldwide,roi_domestic,roi_worldwide,clean_profit


In [66]:
clean_director_info['genres'].value_counts()[:-10]

Documentary              27893
Drama                    20023
Comedy                    8455
Horror                    3996
Comedy,Drama              3399
                         ...  
Comedy,History,Sport         1
Fantasy,War                  1
Action,Music                 1
Comedy,Sport,Thriller        1
Biography,Reality-TV         1
Name: genres, Length: 1053, dtype: int64

In [67]:
clean_director_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131600 entries, 18 to 991700
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   movie_id       131600 non-null  object 
 1   primary_title  131600 non-null  object 
 2   genres         128871 non-null  object 
 3   person_id      131600 non-null  object 
 4   primary_name   131600 non-null  object 
 5   death_year     0 non-null       float64
dtypes: float64(1), object(5)
memory usage: 7.0+ MB
